In [1]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def get_npy_paths(path):
    All_npys_paths = []
    labels = []

    for char_dir in os.listdir(path):
        char_dir_path = os.path.join(path, char_dir)
        for char_npy in os.listdir(char_dir_path):
            char_npy_path = os.path.join(char_dir_path, char_npy)
            All_npys_paths.append(char_npy_path)
            labels.append(char_dir)
            
    return All_npys_paths, labels

def PathToNpy(npy_paths):
    data = np.zeros((len(npy_paths), 21*2))
    for i, npy_path in enumerate(tqdm(npy_paths)) :
        landmarks = np.load(npy_path)
        data[i] = landmarks[:, :2].flatten()
    return data

In [3]:
dataset_path = r"npy/dataset/path"

le = LabelEncoder()

npys_paths, labels = get_npy_paths(dataset_path)
npys = PathToNpy(npys_paths)
labels = le.fit_transform(labels)

100%|██████████| 9551/9551 [02:19<00:00, 68.33it/s] 


In [4]:
x_train, x_test, y_train, y_test = train_test_split(npys, labels, test_size=0.3, random_state=42, shuffle=True)

In [45]:
svc = SVC(C=15, probability=True)
svc.fit(x_train, y_train)

y_train_pred = svc.predict(x_train)
y_test_pred = svc.predict(x_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("train_acc: ", train_acc*100)
print("test_acc: ", test_acc*100)
print("diff: ", (train_acc - test_acc)*100)

train_acc:  98.69857890800299
test_acc:  97.27843684577809
diff:  1.420142062224894


# Trash

In [35]:
r = -1
(1/(1-r)-0.5)*2

0.0

In [ ]:
(1-r**n)/(1-r)

In [42]:
def inf_kernel(sample1:np.ndarray, sample2:np.ndarray, n=1e10):
    sample1 = sample1/np.linalg.norm(sample1)
    sample2 = sample2/np.linalg.norm(sample2)
    r =  np.dot(sample1, sample2)  # cos distance between samples
    r = (r-1)/2
    similarity = (1-r**n)/(1-r) if r !=1 else n     # RBF kernel formula
    return similarity

def get_gram(x1, x2, kernel):
    return np.array([[kernel(_x1, _x2) for _x2 in x2] for _x1 in x1])

def sim(r, gamma=1, n=1e10):
    r = (r-1)/2 # rescale cos similarity to to be from -1, 0
    similarity = (1/(1-r)-0.5)*2
    return similarity**gamma
    # return np.exp(r*gamma)

def get_gram(x1, x2, kernel):
    x1_norm = x1/np.linalg.norm(x1, axis=1).reshape(-1, 1)
    x2_norm = x2/np.linalg.norm(x2, axis=1).reshape(-1, 1)
    return kernel(x1_norm@x2_norm.T, gamma= 2) # (1/(x_train.var() * 42))%10

In [44]:
svc = SVC(C=7, kernel='precomputed')
svc.fit(get_gram(x_train, x_train, sim), y_train)

y_train_pred = svc.predict(get_gram(x_train, x_train, sim))
y_test_pred = svc.predict(get_gram(x_test, x_train, sim))

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("train_acc: ", train_acc*100)
print("test_acc: ", test_acc*100)
print((train_acc - test_acc)*100)

train_acc:  98.44427823485415
test_acc:  97.27843684577809
1.165841389076061


In [48]:
svc = SVC(C=15)
x_train_norm = x_train # /np.linalg.norm(x_train, axis=1).reshape(-1, 1)
x_test_norm = x_test # /np.linalg.norm(x_test, axis=1).reshape(-1, 1)
svc.fit(x_train_norm, y_train)

y_train_pred = svc.predict(x_train_norm)
y_test_pred = svc.predict(x_test_norm)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("train_acc: ", train_acc)
print("test_acc: ", test_acc)
print((train_acc - test_acc)*100)

train_acc:  0.9869857890800299
test_acc:  0.9727843684577809
1.420142062224894
